In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import configparser
import sys

# تابع بررسی مطلوب بودن دقت و تابع هدف
def is_desired_performance(acc, loss, desired_acc, desired_loss):
    return acc >= desired_acc and loss <= desired_loss

# خواندن تنظیمات از فایل کانفیگ
config = configparser.ConfigParser()
config.read('/content/config.ini')

# تعریف مدل شبکه عصبی
class NeuralNet(nn.Module):
    def __init__(self, input_size, output_size, num_layers, num_neurons):
        super(NeuralNet, self).__init__()
        layers = [nn.Linear(input_size, num_neurons)]
        layers.extend([nn.Linear(num_neurons, num_neurons) for _ in range(num_layers - 2)])
        layers.append(nn.Linear(num_neurons, output_size))
        self.fc_layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.fc_layers(x)

# تعریف تابع آموزش و ارزیابی مدل
def train_and_evaluate_model():
    # تعریف تنظیمات اولیه
    global objective_function 
    objective_function = config['Model']['objective_function']
    
    global num_layers
    num_layers = int(config['Model']['num_layers'])
    
    global num_neurons
    num_neurons = int(config['Model']['num_neurons'])
    
    global num_classes
    num_classes = int(config['Model']['num_classes'])
    
    global input_size 
    input_size= int(config['Model']['input_size'])
    
    global epochs 
    epochs= int(config['Training']['epochs'])
    
    global optimizer_type
    optimizer_type = config['Training']['optimizer_type']
    
    global learning_rate
    learning_rate = float(config['Training']['learning_rate'])
    
    global desired_acc 
    desired_acc= float(config['Training']['desired_accuracy'])
    
    global desired_loss
    desired_loss = float(config['Training']['desired_loss'])

    # تعریف مدل
    model = NeuralNet(input_size, num_classes, num_layers, num_neurons)

    # تابع هدف
    if objective_function == 'cross_entropy':
        criterion = nn.CrossEntropyLoss()
    else:
        criterion = nn.MSELoss()

    # بهینه‌ساز
    if optimizer_type == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    elif optimizer_type == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    else:
        raise ValueError("Optimizer type not supported.")

    # حلقه تکرار
    for epoch in range(epochs):
        # تمرین مدل
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # محاسبه دقت
        _, predicted = torch.max(outputs, 1)
        correct = (predicted == labels).sum().item()
        accuracy = correct / len(labels)

        # بررسی مطلوب بودن دقت و تابع هدف
        if is_desired_performance(accuracy, loss.item(), desired_acc, desired_loss):
            break

    return model, accuracy, loss.item()

# داده‌های آموزش و برچسب‌ها
inputs = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
labels = torch.tensor([0, 1, 1, 0], dtype=torch.long)

# آموزش و ارزیابی اولیه مدل
model, accuracy, loss = train_and_evaluate_model()

# بررسی مطلوب بودن دقت و تابع هدف
if not is_desired_performance(accuracy, loss, desired_acc, desired_loss):
    # تغییر تنظیمات برای بهبود نتایج
    config.set('Model', 'num_layers', str(num_layers + 1))
    config.set('Model', 'num_neurons', str(num_neurons + 10))

    with open('config.ini', 'w') as configfile:
        config.write(configfile)

    # اجرای مجدد کد
    exec(open(__file__).read())

# نتایج و تغییرات
print("Accuracy:", accuracy)
print("Loss:", loss)
sys.exit()


Accuracy: 0.5
Loss: 0.6931471824645996
